# OpenAI Evalsを始める

OpenAI Evalsは、言語モデルの性能を評価するためのフレームワークです。このガイドでは、基本的な使用方法から始めて、カスタム評価の作成まで説明します。

## インストール

まず、OpenAI Evalsをインストールします：

```bash
pip install evals
```

## 基本的な使用方法

### 1. 既存の評価を実行する

利用可能な評価を確認するには：

```bash
oaieval list
```

評価を実行するには：

```bash
oaieval gpt-3.5-turbo test-match
```

### 2. 評価結果の確認

評価結果は`/tmp/evallogs/`ディレクトリに保存されます。結果を表示するには：

```bash
oaieval gpt-3.5-turbo test-match --log_to_file /path/to/logfile.jsonl
```

## カスタム評価の作成

### 評価ファイルの構造

評価は通常、以下の構造を持ちます：

```yaml
id: my-custom-eval
description: "カスタム評価の説明"
disclaimer: "この評価は例示目的です"
metrics: [accuracy]
```

### サンプルデータの準備

評価データは`jsonl`形式で準備します：

```json
{"input": "質問1", "ideal": "正解1"}
{"input": "質問2", "ideal": "正解2"}
```

### 評価クラスの実装

カスタム評価クラスを作成します：

```python
import evals
from evals.api import CompletionFn
from evals.elsuite.basic.match import Match

class MyCustomEval(evals.Eval):
    def __init__(self, completion_fns: list[CompletionFn], *args, **kwargs):
        super().__init__(completion_fns, *args, **kwargs)
        
    def eval_sample(self, sample, *_):
        prompt = sample["input"]
        result = self.completion_fn(
            prompt=prompt,
            temperature=0.0,
        )
        
        return evals.record_and_check_match(
            prompt=prompt,
            sampled=result.get_completions()[0],
            expected=sample["ideal"]
        )
```

## 高度な機能

### メトリクスのカスタマイズ

独自のメトリクスを定義できます：

```python
def custom_accuracy(samples):
    correct = sum(1 for sample in samples if sample["match"])
    return {"accuracy": correct / len(samples)}
```

### バッチ処理

大量のデータを効率的に処理するには：

```python
def run_eval_batch(self, samples):
    results = []
    for batch in self.get_batches(samples, batch_size=10):
        batch_results = self.process_batch(batch)
        results.extend(batch_results)
    return results
```

## ベストプラクティス

1. **明確な評価基準**: 評価基準を明確に定義する
2. **多様なテストケース**: 様々なシナリオをカバーする
3. **再現可能性**: 同じ条件で同じ結果が得られるようにする
4. **継続的な改善**: 評価結果を基にモデルを改善する

## トラブルシューティング

### よくある問題

- **API制限**: レート制限に注意し、適切な間隔を設ける
- **メモリ不足**: 大きなデータセットの場合はバッチ処理を使用
- **設定エラー**: 環境変数とAPIキーが正しく設定されているか確認

### デバッグのヒント

```bash
# 詳細なログを有効にする
export EVALS_LOG_LEVEL=DEBUG

# 特定の評価をデバッグモードで実行
oaieval --debug gpt-3.5-turbo my-eval
```

これで、OpenAI Evalsの基本的な使用方法から高度な機能まで理解できました。独自の評価を作成して、言語モデルの性能を効果的に測定しましょう。

**注意：OpenAIは現在、APIを備えたホスト型evalsプロダクトを提供しています！代わりにこちらの使用を推奨します。
[Evals](https://platform.openai.com/docs/guides/evals)をご覧ください**

[OpenAI Evals](https://github.com/openai/evals/tree/main)フレームワークは以下で構成されています：
1. LLM（大規模言語モデル）またはLLM上に構築されたシステムを評価するためのフレームワーク
2. 挑戦的なevalsのオープンソースレジストリ

このノートブックでは以下を扱います：
* 評価と[OpenAI Evals](https://github.com/openai/evals/tree/main)ライブラリの紹介
* Evalの構築
* Evalの実行

#### 評価/`evals`とは何か？

評価とは、LLMアプリケーションが生成する出力を検証・テストするプロセスです。強力な評価（「evals」）を持つことで、コードやモデルの変更に対して堅牢で、より安定した信頼性の高いアプリケーションを実現できます。evalは、LLMまたはLLMシステムの出力品質を測定するために使用されるタスクです。入力プロンプトが与えられると、出力が生成されます。この出力を理想的な回答のセットと照らし合わせて評価し、LLMシステムの品質を判定します。

#### 評価の重要性

`GPT-4`のような基盤モデルを使用して構築している場合、高品質なevalsを作成することは、最もインパクトのある取り組みの一つです。AI ソリューションの開発には反復的な設計プロセスが含まれます。[evalsがないと、異なるモデルバージョンやプロンプトがユースケースにどのような影響を与えるかを理解することが非常に困難で時間がかかる場合があります](https://youtu.be/XGJNo8TpuVA?feature=shared&t=1089)。

OpenAIの[継続的なモデルアップグレード](https://platform.openai.com/docs/models/continuous-model-upgrades)により、evalsを使用することで、ユースケースに対するモデルパフォーマンスを標準化された方法で効率的にテストできます。目標に合わせてカスタマイズされたevalsスイートを開発することで、新しいモデルがユースケースでどのようなパフォーマンスを発揮するかを迅速かつ効果的に理解できます。また、evalsをCI/CDパイプラインの一部にして、デプロイ前に望ましい精度を確実に達成することも可能です。

#### evalsの種類

完了を評価/採点する主な方法は2つあります：コードで検証ロジックを書く方法と、モデル自体を使用して回答を検査する方法です。それぞれを例とともに紹介します。

**回答チェックのためのロジック記述**

最もシンプルで一般的なタイプのevalは、入力と理想的な応答または回答を持ちます。例えば、入力が「オバマが初めて大統領に選出されたのは何年ですか？」で、理想的な回答が「2008」であるevalサンプルを持つことができます。入力をモデルに与えて完了を得ます。モデルが「2008」と言った場合、正解として採点されます。完了に「2008」という語句が含まれているかをチェックする文字列マッチを書くことができます。含まれていれば、正解と見なします。

有効なJSONを生成する入力を持つ別のevalを考えてみましょう：完了をJSONとして解析しようとするコードを書き、解析可能であれば完了を正解と見なすことができます。

**モデル採点：モデルがまず質問に回答し、次にモデルに応答を見て正解かどうかをチェックしてもらう2段階プロセス。**

モデルに面白いジョークを書くよう求める入力を考えてみましょう。モデルが完了を生成します。次に、完了を含む「次のジョークは面白いですか？まず段階的に理由を述べ、次にyes または no で答えてください」という質問に答えるための新しい入力をモデルに作成します。新しいモデルの完了が「yes」で終わる場合、元の完了を正解と見なします。

モデル採点は、`GPT-4`のような最新で最も強力なモデルで最もよく機能し、判断を下す前に推論する能力を与える場合に効果的です。モデル採点にはエラー率があるため、evalsを大規模に実行する前に人間による評価でパフォーマンスを検証することが重要です。最良の結果を得るには、完了を行ったモデルとは異なるモデルを採点に使用することが理にかなっています。例えば、`GPT-4`を使用して`GPT-3.5`の回答を採点するなどです。

#### OpenAI Evalテンプレート

evalsを使用する中で、多くの異なるベンチマークに対応するいくつかの「テンプレート」を発見しました。新しいevalsの開発を簡素化するため、OpenAI Evalsライブラリにこれらのテンプレートを実装しました。例えば、すぐに使用できる2種類のevalテンプレートを定義しています：

* **基本Evalテンプレート**：出力を理想的な回答と比較する決定論的関数を含みます。多肢選択問題への回答や単純で直接的な回答を持つ簡単な質問など、望ましいモデル応答のバリエーションが非常に少ない場合に、以下のテンプレートが有用であることがわかりました。

* **モデル採点テンプレート**：LLMが出力を理想的な回答と比較し、精度を判定しようとする関数を含みます。オープンエンドな質問への回答など、望ましいモデル応答に大きなバリエーションが含まれる可能性がある場合、モデルに自己採点させることが自動評価の実行可能な戦略であることがわかりました。

### セットアップ

まず、[github.com/openai/evals](https://github.com/openai/evals)にアクセスし、`git clone git@github.com:openai/evals.git`でリポジトリをクローンして、[セットアップ手順](https://github.com/openai/evals)を実行してください。

このノートブックで後ほどevalsを実行するには、OpenAI APIキーを設定して指定する必要があります。APIキーを取得した後、`OPENAI_API_KEY`環境変数を使用して指定してください。

evalsを実行する際のAPI使用に関連するコストにご注意ください。

In [1]:
from openai import OpenAI
import pandas as pd

client = OpenAI()

## OpenAI Evalsフレームワーク用の評価の構築

基本的に、evalはデータセットとYAMLファイルで定義されるevalクラスです。evalの作成を開始するには、以下が必要です：

1. `jsonl`形式のテストデータセット
2. 使用するevalテンプレート

### evalデータセットの作成
構文的に正しいSQLを生成するモデルの能力を評価するユースケース用のデータセットを作成しましょう。このユースケースでは、自動車製造に関連する一連のテーブルがあります。

まず、評価したいシステムプロンプトを作成する必要があります。モデルへの指示とテーブル構造の概要を渡します：
```
"TASK: Answer the following question with syntactically correct SQLite SQL. The SQL should be correct and be in context of the previous question-answer pairs.\nTable car_makers, columns = [*,Id,Maker,FullName,Country]\nTable car_names, columns = [*,MakeId,Model,Make]\nTable cars_data, columns = [*,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year]\nTable continents, columns = [*,ContId,Continent]\nTable countries, columns = [*,CountryId,CountryName,Continent]\nTable model_list, columns = [*,ModelId,Maker,Model]\nForeign_keys = [countries.Continent = continents.ContId,car_makers.Country = countries.CountryId,model_list.Maker = car_makers.Id,car_names.Model = model_list.Model,cars_data.Id = car_names.MakeId]"
```

このプロンプトに対して、特定の質問をすることができます：
```
"Q: how many car makers are their in germany?"
```

そして期待される回答があります：
```
"A: SELECT count ( * )  FROM CAR_MAKERS AS T1 JOIN COUNTRIES AS T2 ON T1.Country   =   T2.CountryId WHERE T2.CountryName   =   'germany'"
```

データセットは以下の形式である必要があります：
```
"input": [{"role": "system", "content": "<input prompt>"}, {"role": "user", "content": <user input>}, "ideal": "correct answer"]
```

すべてをまとめると、以下のようになります：
```
{"input": [{"role": "system", "content": "TASK: Answer the following question with syntactically correct SQLite SQL. The SQL should be correct and be in context of the previous question-answer pairs.\nTable car_makers, columns = [*,Id,Maker,FullName,Country]\nTable car_names, columns = [*,MakeId,Model,Make]\nTable cars_data, columns = [*,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year]\nTable continents, columns = [*,ContId,Continent]\nTable countries, columns = [*,CountryId,CountryName,Continent]\nTable model_list, columns = [*,ModelId,Maker,Model]\nForeign_keys = [countries.Continent = continents.ContId,car_makers.Country = countries.CountryId,model_list.Maker = car_makers.Id,car_names.Model = model_list.Model,cars_data.Id = car_names.MakeId]\n"}, {"role": "system", "content": "Q: how many car makers are their in germany"}, "ideal": ["A: SELECT count ( * )  FROM CAR_MAKERS AS T1 JOIN COUNTRIES AS T2 ON T1.Country   =   T2.CountryId WHERE T2.CountryName   =   'germany'"]}
```

evalデータセットの構築プロセスを高速化する一つの方法は、`GPT-4`を使用して合成データを生成することです。

In [2]:
## Use GPT-4 to generate synthetic data
# Define the system prompt and user input (these should be filled as per the specific use case)
system_prompt = """You are a helpful assistant that can ask questions about a database table and write SQL queries to answer the question.
    A user will pass in a table schema and your job is to return a question answer pairing. The question should relevant to the schema of the table,
    and you can speculate on its contents. You will then have to generate a SQL query to answer the question. Below are some examples of what this should look like.

    Example 1
    ```````````
    User input: Table museum, columns = [*,Museum_ID,Name,Num_of_Staff,Open_Year]\nTable visit, columns = [*,Museum_ID,visitor_ID,Num_of_Ticket,Total_spent]\nTable visitor, columns = [*,ID,Name,Level_of_membership,Age]\nForeign_keys = [visit.visitor_ID = visitor.ID,visit.Museum_ID = museum.Museum_ID]\n
    Assistant Response:
    Q: How many visitors have visited the museum with the most staff?
    A: SELECT count ( * )  FROM VISIT AS T1 JOIN MUSEUM AS T2 ON T1.Museum_ID   =   T2.Museum_ID WHERE T2.Num_of_Staff   =   ( SELECT max ( Num_of_Staff )  FROM MUSEUM ) 
    ```````````

    Example 2
    ```````````
    User input: Table museum, columns = [*,Museum_ID,Name,Num_of_Staff,Open_Year]\nTable visit, columns = [*,Museum_ID,visitor_ID,Num_of_Ticket,Total_spent]\nTable visitor, columns = [*,ID,Name,Level_of_membership,Age]\nForeign_keys = [visit.visitor_ID = visitor.ID,visit.Museum_ID = museum.Museum_ID]\n
    Assistant Response:
    Q: What are the names who have a membership level higher than 4?
    A: SELECT Name   FROM VISITOR AS T1 WHERE T1.Level_of_membership   >   4 
    ```````````

    Example 3
    ```````````
    User input: Table museum, columns = [*,Museum_ID,Name,Num_of_Staff,Open_Year]\nTable visit, columns = [*,Museum_ID,visitor_ID,Num_of_Ticket,Total_spent]\nTable visitor, columns = [*,ID,Name,Level_of_membership,Age]\nForeign_keys = [visit.visitor_ID = visitor.ID,visit.Museum_ID = museum.Museum_ID]\n
    Assistant Response:
    Q: How many tickets of customer id 5?
    A: SELECT count ( * )  FROM VISIT AS T1 JOIN VISITOR AS T2 ON T1.visitor_ID   =   T2.ID WHERE T2.ID   =   5 
    ```````````
    """

user_input = "Table car_makers, columns = [*,Id,Maker,FullName,Country]\nTable car_names, columns = [*,MakeId,Model,Make]\nTable cars_data, columns = [*,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year]\nTable continents, columns = [*,ContId,Continent]\nTable countries, columns = [*,CountryId,CountryName,Continent]\nTable model_list, columns = [*,ModelId,Maker,Model]\nForeign_keys = [countries.Continent = continents.ContId,car_makers.Country = countries.CountryId,model_list.Maker = car_makers.Id,car_names.Model = model_list.Model,cars_data.Id = car_names.MakeId]"

messages = [{
        "role": "system",
        "content": system_prompt
    },
    {
        "role": "user",
        "content": user_input
    }
]

completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=messages,
    temperature=0.7,
    n=5
)

for choice in completion.choices:
    print(choice.message.content + "\n")


Q: What is the average horsepower for cars made in Europe?
A: SELECT AVG(cars_data.Horsepower) FROM cars_data JOIN car_names ON cars_data.Id = car_names.MakeId JOIN model_list ON car_names.Model = model_list.Model JOIN car_makers ON model_list.Maker = car_makers.Id JOIN countries ON car_makers.Country = countries.CountryId JOIN continents ON countries.Continent = continents.ContId WHERE continents.Continent = 'Europe'

Q: What is the average horsepower for cars made in the USA?
A: SELECT AVG(cars_data.Horsepower) FROM cars_data JOIN car_names ON cars_data.Id = car_names.MakeId JOIN car_makers ON car_names.MakeId = car_makers.Id JOIN countries ON car_makers.Country = countries.CountryId WHERE countries.CountryName = 'USA'

Q: What is the average horsepower for cars produced in countries from the continent with the id '3'?
A: SELECT AVG(cars_data.Horsepower) FROM cars_data JOIN car_names ON cars_data.Id = car_names.MakeId JOIN model_list ON car_names.Model = model_list.Model JOIN car_mak

合成データを取得したら、評価データセットの形式に合わせて変換する必要があります。

In [3]:
eval_data = []
input_prompt = "TASK: Answer the following question with syntactically correct SQLite SQL. The SQL should be correct and be in context of the previous question-answer pairs.\nTable car_makers, columns = [*,Id,Maker,FullName,Country]\nTable car_names, columns = [*,MakeId,Model,Make]\nTable cars_data, columns = [*,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year]\nTable continents, columns = [*,ContId,Continent]\nTable countries, columns = [*,CountryId,CountryName,Continent]\nTable model_list, columns = [*,ModelId,Maker,Model]\nForeign_keys = [countries.Continent = continents.ContId,car_makers.Country = countries.CountryId,model_list.Maker = car_makers.Id,car_names.Model = model_list.Model,cars_data.Id = car_names.MakeId]"

for choice in completion.choices:
    question = choice.message.content.split("Q: ")[1].split("\n")[0]  # Extracting the question
    answer = choice.message.content.split("\nA: ")[1].split("\n")[0]  # Extracting the answer
    eval_data.append({
        "input": [
            {"role": "system", "content": input_prompt},
            {"role": "user", "content": question},
        ],
        "ideal": answer
    })

for item in eval_data:
    print(item)

{'input': [{'role': 'system', 'content': 'TASK: Answer the following question with syntactically correct SQLite SQL. The SQL should be correct and be in context of the previous question-answer pairs.\nTable car_makers, columns = [*,Id,Maker,FullName,Country]\nTable car_names, columns = [*,MakeId,Model,Make]\nTable cars_data, columns = [*,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year]\nTable continents, columns = [*,ContId,Continent]\nTable countries, columns = [*,CountryId,CountryName,Continent]\nTable model_list, columns = [*,ModelId,Maker,Model]\nForeign_keys = [countries.Continent = continents.ContId,car_makers.Country = countries.CountryId,model_list.Maker = car_makers.Id,car_names.Model = model_list.Model,cars_data.Id = car_names.MakeId]'}, {'role': 'user', 'content': 'What is the average horsepower for cars made in Europe?'}], 'ideal': "SELECT AVG(cars_data.Horsepower) FROM cars_data JOIN car_names ON cars_data.Id = car_names.MakeId JOIN model_list ON car_names.Model 

次に、フレームワークで実行するためのevalレジストリを作成する必要があります。

evalsフレームワークには、以下のプロパティで構成された`.yaml`ファイルが必要です：
* `id` - evalの識別子
* `description` - evalの短い説明
* `disclaimer` - evalに関する追加の注意事項
* `metrics` - 選択できるevalメトリクスには3つのタイプがあります：match、includes、fuzzyMatch

私たちのevalでは、以下のように設定します：

In [4]:
"""
spider-sql:
  id: spider-sql.dev.v0
  metrics: [accuracy]
  description: Eval that scores SQL code from 194 examples in the Spider Text-to-SQL test dataset. The problems are selected by taking the first 10 problems for each database that appears in the test set.
    Yu, Tao, et al. \"Spider; A Large-Scale Human-Labeled Dataset for Complex and Cross-Domain Semantic Parsing and Text-to-SQL Task.\" Proceedings of the 2018 Conference on Empirical Methods in Natural Language Processing, 2018, https://doi.org/10.18653/v1/d18-1425.
  disclaimer: Problems are solved zero-shot with no prompting other than the schema; performance may improve with training examples, fine tuning, or a different schema format. Evaluation is currently done through model-grading, where SQL code is not actually executed; the model may judge correct SQL to be incorrect, or vice-versa.
spider-sql.dev.v0:
  class: evals.elsuite.modelgraded.classify:ModelBasedClassify
  args:
    samples_jsonl: sql/spider_sql.jsonl
    eval_type: cot_classify
    modelgraded_spec: sql
  """""

'\nspider-sql:\n  id: spider-sql.dev.v0\n  metrics: [accuracy]\n  description: Eval that scores SQL code from 194 examples in the Spider Text-to-SQL test dataset. The problems are selected by taking the first 10 problems for each database that appears in the test set.\n    Yu, Tao, et al. "Spider; A Large-Scale Human-Labeled Dataset for Complex and Cross-Domain Semantic Parsing and Text-to-SQL Task." Proceedings of the 2018 Conference on Empirical Methods in Natural Language Processing, 2018, https://doi.org/10.18653/v1/d18-1425.\n  disclaimer: Problems are solved zero-shot with no prompting other than the schema; performance may improve with training examples, fine tuning, or a different schema format. Evaluation is currently done through model-grading, where SQL code is not actually executed; the model may judge correct SQL to be incorrect, or vice-versa.\nspider-sql.dev.v0:\n  class: evals.elsuite.modelgraded.classify:ModelBasedClassify\n  args:\n    samples_jsonl: sql/spider_sql.js

## 評価の実行

`oaieval` CLIを使用してこの評価を実行できます。セットアップするには、ライブラリをインストールしてください：`pip install .`（[OpenAI Evalsライブラリ](github.com/openai/evals)をローカルで実行している場合）または、既存の評価を実行している場合は`pip install oaieval`を使用してください。

次に、CLIを使用して評価を実行します：`oaieval gpt-3.5-turbo spider-sql`

このコマンドはモデル名と評価セット名を期待します。2つのコマンドラインインターフェース（CLI）を提供していることに注意してください：単一の評価を実行するための`oaieval`と、評価のセットを実行するための`oaievalset`です。有効な評価名は`evals/registry/evals`下のYAMLファイルで指定されており、対応する実装は`evals/elsuite`で見つけることができます。

In [5]:
!pip install evals --quiet

`oaieval` CLIは、デフォルトの動作を変更するために様々なフラグを受け入れることができます。`oaieval --help`を実行すると、CLIオプションの完全なリストを確認できます。

そのコマンドを実行した後、精度の最終レポートがコンソールに出力され、完全なレポートを含む一時ファイルのファイルパスも表示されます。

`oaieval`は、上記のセル4で指定された形式に従って、`evals/registry/evals`ディレクトリ内の`spider-sql`評価YAMLファイルを検索します。評価データセットへのパスは、評価YAMLファイル内のargs:パラメータの下で`samples_jsonl: sql/spider_sql.jsonl`として指定されており、ファイルの内容はJSONL形式（上記のステップ3で生成されたもの）になっています。

そのコマンドを実行すると、精度の最終レポートがコンソールに出力され、完全なレポートを含む一時ファイルへのファイルパスも表示されます。

In [6]:
!oaieval gpt-3.5-turbo spider-sql --max_samples 25

[2024-03-26 19:44:39,836] [registry.py:257] Loading registry from /Users/shyamal/.virtualenvs/openai/lib/python3.11/site-packages/evals/registry/evals
[2024-03-26 19:44:43,623] [registry.py:257] Loading registry from /Users/shyamal/.evals/evals
[2024-03-26 19:44:43,635] [oaieval.py:189] Run started: 240327024443FACXGMKA
[2024-03-26 19:44:43,663] [registry.py:257] Loading registry from /Users/shyamal/.virtualenvs/openai/lib/python3.11/site-packages/evals/registry/modelgraded
[2024-03-26 19:44:43,851] [registry.py:257] Loading registry from /Users/shyamal/.evals/modelgraded
[2024-03-26 19:44:43,853] [data.py:90] Fetching /Users/shyamal/.virtualenvs/openai/lib/python3.11/site-packages/evals/registry/data/sql/spider_sql.jsonl
[2024-03-26 19:44:43,878] [eval.py:36] Evaluating 25 samples
[2024-03-26 19:44:43,952] [eval.py:144] Running in threaded mode with 10 threads!
  0%|                                                    | 0/25 [00:00<?, ?it/s][2024-03-26 19:44:44,810] [_client.py:1026] H

`oaievalset`はモデル名と評価セット名を期待しており、有効なオプションは`evals/registry/eval_sets`配下のYAMLファイルで指定されています。

### evalログの確認

evalログは `/tmp/evallogs` に配置されており、各評価実行ごとに異なるログファイルが作成されます。

In [7]:
log_name = '240327024443FACXGMKA_gpt-3.5-turbo_spider-sql.jsonl' # "EDIT THIS" - copy from above
events = f"/tmp/evallogs/{log_name}"
display(pd.read_json(events, lines=True).head(5))

,spec,final_report,run_id,event_id,sample_id,type,data,created_by,created_at
0,"{'completion_fns': ['gpt-3.5-turbo'], 'eval_name': 'spider-sql.dev.v0', 'base_eval': 'spider-sql', 'split': 'dev', 'run_config': {'completion_fns': ['gpt-3.5-turbo'], 'eval_spec': {'cls': 'evals.elsuite.modelgraded.classify:ModelBasedClassify', 'registry_path': '/Users/shyamal/.virtualenvs/openai/lib/python3.11/site-packages/evals/registry', 'args': {'samples_jsonl': 'sql/spider_sql.jsonl', 'eval_type': 'cot_classify', 'modelgraded_spec': 'sql'}, 'key': 'spider-sql.dev.v0', 'group': 'sql'}, 'seed': 20220722, 'max_samples': 25, 'command': '/Users/shyamal/.virtualenvs/openai/bin/oaieval gpt-3.5-turbo spider-sql --max_samples 25', 'initial_settings': {'visible': False}}, 'created_by': '', 'run_id': '240327024443FACXGMKA', 'created_at': '2024-03-27 02:44:43.626043'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,NaN,"{'counts/Correct': 20, 'counts/Incorrect': 5, 'score': 0.8}",NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,NaN,NaN,240327024443FACXGMKA,0.0,spider-sql.dev.88,sampling,"{'prompt': [{'content': 'Answer the following question with syntactically correct SQLite SQL. Be creative but the SQL must be correct. Use only the following tables and columns: Table: players. Columns: player_id (number), first_name (text), last_name (text), hand (text), birth_date (time), country_code (text) Table: matches. Columns: best_of (number), draw_size (number), loser_age (number), loser_entry (text), loser_hand (text), loser_ht (number), loser_id (number), loser_ioc (text), loser_name (text), loser_rank (number), loser_rank_points (number), loser_seed (number), match_num (number), minutes (number), round (text), score (text), surface (text), tourney_date (time), tourney_id (text), tourney_level (text), tourney_name (text), winner_age (number), winner_entry (text), winner_hand (text), winner_ht (number), winner_id (number), winner_ioc (text), winner_name (text), winner_rank (number), winner_rank_points (number), winner_seed (number), year (number) Table: rankings. Columns: ranking_date (time), ranking (number), player_id (number), ranking_points (number), tours (number) Question: Find the average rank of winners in all matches. ', 'role': 'system'}], 'sampled': ['SELECT AVG(winner_rank) AS average_rank_of_winners FROM matches;']}",,2024-03-27 02:44:44.821110+00:00
3,NaN,NaN,240327024443FACXGMKA,1.0,spider-sql.dev.82,sampling,"{'prompt': [{'content': 'Answer the following question with syntactically correct SQLite SQL. Be creative but the SQL must be correct. Use only the following tables and columns: Table: players. Columns: player_id (number), first_name (text), last_name (text), hand (text), birth_date (time), country_code (text) Table: matches. Columns: best_of (number), draw_size (number), loser_age (number), loser_entry (text), loser_hand (text), loser_ht (number), loser_id (number), loser_ioc (text), loser_name (text), loser_rank (number), loser_rank_points (number), loser_seed (number), match_num (number), minutes (number), round (text), score (text), surface (text), tourney_date (time), tourney_id (text), tourney_level (text), tourney_name (text), winner_age (number), winner_entry (text), winner_hand (text), winner_ht (number), winner_id (number), winner_ioc (text), winner_name (text), winner_rank (number), winner_rank_points (number), winner_seed (number), year (number) Table: rankings. Columns: ranking_date (time), ranking (number), player_id (number), ranking_points (number), tours (number) Question: Find the total number of matches. ', 'role': 'system'}], 'sampled': ['SELECT COUNT(*) AS total_matches FROM matches;']}",,2024-03-27 02:44:44.831848+00:00
4,NaN,NaN,240327024443FACXGMKA,2.0,spider-sql.dev.25,sampling,"{'prompt': [{'content': 'Answer the following question with syntactically correct SQLite SQL. Be creative but the SQL must be correct. Use only the following tables and columns: Table: continents. Columns: ContId (number), Continent (text) Table: countries. Columns: Count

In [8]:
# processing the log events generated by oaieval

with open(events, "r") as f:
    events_df = pd.read_json(f, lines=True)

このファイルには評価の構造化されたログが含まれます。最初のエントリでは、完了関数、評価名、実行設定、作成者名、実行ID、作成タイムスタンプを含む評価の詳細な仕様が提供されます。

In [9]:
display(events_df.iloc[0].spec)

{'completion_fns': ['gpt-3.5-turbo'],
 'eval_name': 'spider-sql.dev.v0',
 'base_eval': 'spider-sql',
 'split': 'dev',
 'run_config': {'completion_fns': ['gpt-3.5-turbo'],
  'eval_spec': {'cls': 'evals.elsuite.modelgraded.classify:ModelBasedClassify',
   'registry_path': '/Users/shyamal/.virtualenvs/openai/lib/python3.11/site-packages/evals/registry',
   'args': {'samples_jsonl': 'sql/spider_sql.jsonl',
    'eval_type': 'cot_classify',
    'modelgraded_spec': 'sql'},
   'key': 'spider-sql.dev.v0',
   'group': 'sql'},
  'seed': 20220722,
  'max_samples': 25,
  'command': '/Users/shyamal/.virtualenvs/openai/bin/oaieval gpt-3.5-turbo spider-sql --max_samples 25',
  'initial_settings': {'visible': False}},
 'created_by': '',
 'run_id': '240327024443FACXGMKA',
 'created_at': '2024-03-27 02:44:43.626043'}

評価の最終レポートを提供するエントリも見てみましょう。

In [10]:
display(events_df.dropna(subset=['final_report']).iloc[0]['final_report'])

{'counts/Correct': 20, 'counts/Incorrect': 5, 'score': 0.8}

特定のサンプル（`sample_id`）、結果、イベントタイプ、メタデータを提供する個別の評価イベントも確認できます。

In [11]:
pd.set_option('display.max_colwidth', None)  # None means no truncation
display(events_df.iloc[2][['run_id', 'event_id', 'sample_id', 'type', 'data', 'created_at']])

run_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [12]:
# Inspect samples
for i, row in events_df[events_df['type'] == 'sampling'].head(5).iterrows():
    data = pd.json_normalize(row['data'])
    print(f"Prompt: {data['prompt'].iloc[0]}")
    print(f"Sampled: {data['sampled'].iloc[0]}")
    print("-" * 10)

Prompt: [{'content': 'Answer the following question with syntactically correct SQLite SQL. Be creative but the SQL must be correct.\nUse only the following tables and columns:\nTable: players. Columns: player_id (number), first_name (text), last_name (text), hand (text), birth_date (time), country_code (text)\nTable: matches. Columns: best_of (number), draw_size (number), loser_age (number), loser_entry (text), loser_hand (text), loser_ht (number), loser_id (number), loser_ioc (text), loser_name (text), loser_rank (number), loser_rank_points (number), loser_seed (number), match_num (number), minutes (number), round (text), score (text), surface (text), tourney_date (time), tourney_id (text), tourney_level (text), tourney_name (text), winner_age (number), winner_entry (text), winner_hand (text), winner_ht (number), winner_id (number), winner_ioc (text), winner_name (text), winner_rank (number), winner_rank_points (number), winner_seed (number), year (number)\nTable: rankings. Columns: r

テストが成功しなかった原因を理解するために、失敗を確認してみましょう。

In [13]:
def pretty_print_text(prompt):
    # Define markers for the sections
    markers = {
        "question": "[Question]:",
        "expert": "[Expert]:",
        "submission": "[Submission]:",
        "end": "[END DATA]"
    }
    
    # Function to extract text between markers
    def extract_text(start_marker, end_marker):
        start = prompt.find(start_marker) + len(start_marker)
        end = prompt.find(end_marker)
        text = prompt[start:end].strip()
        if start_marker == markers["question"]:
            text = text.split("\n\nQuestion:")[-1].strip() if "\n\nQuestion:" in text else text
        elif start_marker == markers["submission"]:
            text = text.replace("```sql", "").replace("```", "").strip()
        return text
    
    # Extracting text for each section
    question_text = extract_text(markers["question"], markers["expert"])
    expert_text = extract_text(markers["expert"], markers["submission"])
    submission_text = extract_text(markers["submission"], markers["end"])
    
    # HTML color codes and formatting
    colors = {
        "question": '<span style="color: #0000FF;">QUESTION:<br>', 
        "expert": '<span style="color: #008000;">EXPECTED:<br>',  
        "submission": '<span style="color: #FFA500;">SUBMISSION:<br>' 
    }
    color_end = '</span>'
    
    # Display each section with color
    from IPython.display import display, HTML
    display(HTML(f"{colors['question']}{question_text}{color_end}"))
    display(HTML(f"{colors['expert']}{expert_text}{color_end}"))
    display(HTML(f"{colors['submission']}{submission_text}{color_end}"))

In [14]:
# Inspect metrics where choice is made and print only the prompt, result, and expected result if the choice is incorrect
for i, row in events_df[events_df['type'] == 'metrics'].iterrows():
    if row['data']['choice'] == 'Incorrect':
        # Get the previous row's data, which contains the prompt and the expected result
        prev_row = events_df.iloc[i-1]
        prompt = prev_row['data']['prompt'][0]['content'] if 'prompt' in prev_row['data'] and len(prev_row['data']['prompt']) > 0 else "Prompt not available"
        expected_result = prev_row['data'].get('ideal', 'Expected result not provided')
        
        # Current row's data will be the actual result
        result = row['data'].get('result', 'Actual result not provided')
        
        pretty_print_text(prompt)
        print("-" * 40)

----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------


発生している失敗例を確認すると、以下のことがわかります：
* 2番目の不正解では、'Templates'テーブルとの不要な結合がありました。我々の評価システムはこれを正確に識別し、不正解としてフラグを立てることができました。
* その他のいくつかの回答では、軽微な構文の違いが原因で不正解としてフラグが立てられました。
  * このような状況では、特定のスタイル選択を確実にするためにプロンプトの反復を続けるべきか、それとも評価スイートを修正してこの変動を捉えるべきかを検討する価値があります。
  * この種の失敗は、結果の採点精度を確保する方法として、モデルによる評価の潜在的な必要性を示唆しています。

# 結論

効果的なevalsの構築は、LLMベースのアプリケーションの開発サイクルにおける中核的な部分です。OpenAI Evalsフレームワークは、すぐに使えるevalsを構築するための基本構造を提供し、様々なユースケースに対して新しいテストを素早く立ち上げることを可能にします。このガイドでは、evalを作成し、実行し、結果を分析する方法をステップバイステップで実演しました。

このガイドで示した例は、evalsの直接的なユースケースを表しています。このフレームワークを引き続き探求する際は、実際の本番環境のユースケースに向けて、より複雑なモデル評価型evalsの作成を探求することをお勧めします。評価を楽しんでください！